In [68]:
import pandas as pd

train_df = pd.read_csv('Dataset/Processed_Data_NoSmiley.csv')
train_df.head()

test_df = pd.read_csv('Dataset/4_emo_reduced.csv')
test_df.head()

,Utterances,Label
0,"['That was a joke btw...', 'it was', 'Yes 😁']",happy
1,"['yes, good advice', 'best advice ...', 'i gre...",others
2,"['Nice to meet u', 'Hi, nice to meet you too! ...",happy
3,"['First you hurt me', 'okay', 'So I talked ...",angry
4,['is my grammar perfect or should I need to le...,happy


In [69]:
train_df_bert = pd.DataFrame({
    'id':range(len(train_df)),
    'label':train_df['Label'],
    'alpha':['a']*train_df.shape[0],
    'utterance': train_df['Utterances']
})
train_df_bert.head()

test_df_bert = pd.DataFrame({
    'id':range(len(test_df)),
    'label':test_df['Label'],
    'alpha':['a']*test_df.shape[0],
    'utterance': test_df['Utterances']
})
test_df_bert.head()

,id,label,alpha,utterance
0,0,happy,a,"['That was a joke btw...', 'it was', 'Yes 😁']"
1,1,others,a,"['yes, good advice', 'best advice ...', 'i gre..."
2,2,happy,a,"['Nice to meet u', 'Hi, nice to meet you too! ..."
3,3,angry,a,"['First you hurt me', 'okay', 'So I talked ..."
4,4,happy,a,['is my grammar perfect or should I need to le...


In [70]:
train_df_bert['label'] = train_df_bert['label'].replace(to_replace = "happy", value = "0")
train_df_bert['label'] = train_df_bert['label'].replace(to_replace = "sad", value = "1")
train_df_bert['label'] = train_df_bert['label'].replace(to_replace = "angry", value = "2")
train_df_bert['label'] = train_df_bert['label'].replace(to_replace = "others", value = "3")
train_df_bert.head()

test_df_bert['label'] = test_df_bert['label'].replace(to_replace = "happy", value = "0")
test_df_bert['label'] = test_df_bert['label'].replace(to_replace = "sad", value = "1")
test_df_bert['label'] = test_df_bert['label'].replace(to_replace = "angry", value = "2")
test_df_bert['label'] = test_df_bert['label'].replace(to_replace = "others", value = "3")
test_df_bert.head()

,id,label,alpha,utterance
0,0,0,a,"['That was a joke btw...', 'it was', 'Yes 😁']"
1,1,3,a,"['yes, good advice', 'best advice ...', 'i gre..."
2,2,0,a,"['Nice to meet u', 'Hi, nice to meet you too! ..."
3,3,2,a,"['First you hurt me', 'okay', 'So I talked ..."
4,4,0,a,['is my grammar perfect or should I need to le...


In [72]:
train_df_bert['utterance'] = train_df_bert['utterance'].replace(to_replace = ['\[', '\]'], value = '', regex = True)
train_df_bert.head()

test_df_bert['utterance'] = test_df_bert['utterance'].replace(to_replace = ['\[', '\]'], value = '', regex = True)
test_df_bert.head()

,id,label,alpha,utterance
0,0,0,a,"'That was a joke btw...', 'it was', 'Yes 😁'"
1,1,3,a,"'yes, good advice', 'best advice ...', 'i grea..."
2,2,0,a,"'Nice to meet u', 'Hi, nice to meet you too! 😸..."
3,3,2,a,"'First you hurt me', 'okay', 'So I talked r..."
4,4,0,a,'is my grammar perfect or should I need to lea...


Name file train.tsv and dev.tsv as BERT comes with data loading classes that expects train and dev files in the above format.
1. Train file for training.
2. Dev file for evaluation.

In [75]:
train_df_bert.to_csv('Dataset/Bert/train.tsv', sep='\t', index=False, header=False)
test_df_bert.to_csv('Dataset/Bert/dev.tsv', sep='\t', index=False, header=False)

In [76]:
import torch
import pickle
from torch.utils.data import (DataLoader, RandomSampler, SequentialSampler, TensorDataset)
from torch.nn import CrossEntropyLoss, MSELoss

from tqdm import tqdm_notebook, trange
import os
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM, BertForSequenceClassification
from pytorch_pretrained_bert.optimization import BertAdam, WarmupLinearSchedule

from multiprocessing import Pool, cpu_count
from tools import *
import convert_examples_to_features

# OPTIONAL: if you want to have more information on what's happening, activate the logger as follows
import logging
logging.basicConfig(level=logging.INFO)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

ModuleNotFoundError: No module named 'convert_examples_to_features'